In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

# Generar los DataFrames

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
ventas_año = ventas_año[ventas_año['invoice_date'].dt.month < 7]

In [4]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [5]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

productos_sin_compra['tolerance_order_date'] = productos_sin_compra['order_date']

In [6]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [7]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [8]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [9]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [10]:
compras_odoo = pd.merge(
                    compras_linea,
                    compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'
                )

compras_odoo['tolerance_order_date'] = compras_odoo['order_date'].dt.normalize() - pd.Timedelta(days=5)

cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras_odoo[cols_to_Int64] = compras_odoo[cols_to_Int64].astype('Int64')

compras_odoo['product_qty'] = compras_odoo['product_qty'].astype('Float64')
compras_odoo['vendedora'] = compras_odoo['vendedora'].convert_dtypes()

compras = pd.concat([compras_odoo, productos_sin_compra])

# Checks

In [11]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')


Hay proveedores no calificados


,order_id,order_name,order_state,order_date,partner_id,partner_name,partner_fact_ref,partner_fact_date,capturista,vendedora
14,3287,P03272,purchase,2024-07-03 23:13:21,14576,FYMSA BAJA SUR,SJ 33643,2024-07-03,Elsa Ivette Diaz Leyva,Yamilet Blanco Salas


In [12]:
check_costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        direction = 'backward')
)

check2 = check_costo_venta[check_costo_venta['product_price'].isna()][['product_id', 'product_name_x']]

print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 21 renglones sin costo de la venta.
Los productos sin costo son los siguientes:


,product_id,product_name_x
10061,14757,Nuevo *0*
10282,28586,Anticipo
13415,28638,Servicios de Facturación
18005,4,Anticipo (PdV)


# Preliminares... obtener los segmentos a trabajar

Sacar los productos que se compran con varios proveedores

In [13]:
prod_comprados_odoo = ( compras_odoo[['partner_id', 'partner_name', 'product_id_pp', 'product_name']]
                        .sort_values('partner_id')
                        .groupby('product_id_pp')
                        .agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
                    ).reset_index()

prod_comprados_odoo['diff'] = prod_comprados_odoo[('partner_id', 'mean')]  == prod_comprados_odoo[('partner_id', 'first')]

ids_productos_varios_prov = prod_comprados_odoo.loc[prod_comprados_odoo['diff'] == False]['product_id_pp']
ids_productos_unicos_prov = prod_comprados_odoo.loc[prod_comprados_odoo['diff'] == True]['product_id_pp']

print(f"Se han comprado un total de {len(prod_comprados_odoo['product_id_pp'])} productos en Odoo:")
print(f'    - {len(ids_productos_varios_prov)} productos con más de un proveedor', f"{len(ids_productos_varios_prov)/len(prod_comprados_odoo['product_id_pp'])*100:.2f}%")
print(f'    - {len(ids_productos_unicos_prov)} productos con un único proveedor', f"{len(ids_productos_unicos_prov)/len(prod_comprados_odoo['product_id_pp'])*100:.2f}%")

Se han comprado un total de 5120 productos en Odoo:
    - 224 productos con más de un proveedor 4.38%
    - 4896 productos con un único proveedor 95.62%


De los productos comprados a más de un proveedor, se obtienen los que no han cambiado su costo en el tiempo

In [14]:
compras_prod_varios_prov = compras_odoo[compras_odoo['product_id_pp'].isin(ids_productos_varios_prov)]


compras_prod_varios_prov_agrupado = (compras_prod_varios_prov
                                     .sort_values('partner_id')
                                     .groupby('product_id_pp')
                                     .agg({'product_name':['first'], 'line_id':['count'], 'product_price':['first', 'mean']})
                                    ).reset_index()

compras_prod_varios_prov_agrupado['diff'] = compras_prod_varios_prov_agrupado[('product_price', 'first')] == compras_prod_varios_prov_agrupado[('product_price', 'mean')]

ids_prod_varios_prov_costo_sin_cambios = compras_prod_varios_prov_agrupado.loc[compras_prod_varios_prov_agrupado['diff'] == True]['product_id_pp']
ids_prod_varios_prov_costo_con_cambios = compras_prod_varios_prov_agrupado.loc[compras_prod_varios_prov_agrupado['diff'] == False]['product_id_pp']


print(f'De los {len(ids_productos_varios_prov)} productos comprados con varios proveedores:')
print(f'    - {len(ids_prod_varios_prov_costo_sin_cambios)} productos no han cambiado su costo, representan a un {len(ids_prod_varios_prov_costo_sin_cambios)/len(ids_productos_varios_prov)*100:.2f}%')
print(f'    - {len(ids_prod_varios_prov_costo_con_cambios)} productos sí han cambiado su costo, representan a un {len(ids_prod_varios_prov_costo_con_cambios)/len(ids_productos_varios_prov)*100:.2f}%')


De los 224 productos comprados con varios proveedores:
    - 7 productos no han cambiado su costo, representan a un 3.12%
    - 217 productos sí han cambiado su costo, representan a un 96.88%


### Productos fase A del costo de ventas

Se concatena los productos comprados a un único proveedor ('ids_productos_unicos_prov') y los productos comprados a varios proveedores que no han cambiado su precio ('ids_prod_varios_prov_costo_sin_cambios')  

In [15]:
ids_prod_fase_A = pd.concat([ids_productos_unicos_prov, ids_prod_varios_prov_costo_sin_cambios]).sort_values()

Se definen los dataframes de ventas y compras para la fase A, donde al ser productos de venta "diaria" se utiliza el data frame de compras en general.

In [16]:
ventas_fase_A = ventas_año[ventas_año['product_id'].isin(ids_prod_fase_A)]
compras_fase_A = compras[compras['product_id_pp'].isin(ids_prod_fase_A)]

### Productos fase B del costo de ventas

Resto de los productos que no pertenecen a la fase A.  
Se definen los dataframes de ventas y compras para la fase B, donde al ser productos de venta "especial" se utiliza el data frame de compras_odoo.

In [17]:
ventas_fase_B = ventas_año[~ventas_año['product_id'].isin(ids_prod_fase_A)]
compras_fase_B = compras_odoo[~compras_odoo['product_id_pp'].isin(ids_prod_fase_A)]

# Fase A: punto focal "Líneas de venta"
## Productos comprados con un único proveedor y
## Productos comprados a varios proveedores que no cambiaron su precio en el tiempo

Se obtiene el costo de venta en el dataframe 'ventas_año' donde  los renglones de venta coinciden con los 'ids_prod_fase_A', su contraparte es el total de las compras.
Estas ventas no son de tipo "especial", sino venta normal y se trata de encontrar un "Último costo". Es por esto que no se usa tolerancia y la dirección es "backward".

In [18]:
costo_venta_fase_A = (
    pd.merge_asof(
        left = ventas_fase_A.sort_values('invoice_date'),
        right = compras_fase_A.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        direction = 'backward')
)

# Fase B: punto focal "Líneas de compra"
## Productos comprados a varios proveedores que sí cambiaron su precio en el tiempo

### 1er vuelta: productos que con un match sencillo, coinciden en 3 condiciones [producto, vendedora, cantidad]

Se estudian las líneas de compras con 'ids_prod_fase_B'

In [19]:
print(f'Son un total de {len(compras_odoo)} líneas de compra en Odoo. (Aquí no hay líneas del SAE)')

print(f'\nEn la fase B, quedan sólo {len(compras_fase_B)} de ellas {len(compras_fase_B)/len(compras_odoo)*100:.2f}%')

Son un total de 13188 líneas de compra en Odoo. (Aquí no hay líneas del SAE)

En la fase B, quedan sólo 1429 de ellas 10.84%


Debido a que el punto focal son las compras, no se puede hablar de un costo de la venta, sino de un "match entre ids".  
Se obtiene el match de líneas de compra de fase B donde  los renglones de coinciden con los 'ids_productos_varios_prov_costo_con_cambios'. Su contraparte es el total de ventas_año con ids de producto de fase B.  
Debido a que estas compras son para ventas de tipo "Especial", se trata de buscar una venta exacta con las 3 condiciones descritas anteriormente. Se usa una tolerancia y con dirección "nearest".

In [20]:
match_1ro = (
    pd.merge_asof(
        left = compras_fase_B.sort_values('order_date'), 
        right = ventas_fase_B.sort_values('invoice_date'),
        
        left_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        right_by = ['product_id', 'salesperson_name', 'quantity'], 
        
        left_on = 'order_date', 
        right_on = 'invoice_date', 

        tolerance = pd.Timedelta(days=5),
        
        direction = 'nearest')
)

Se descartan del merge anterior los match donde hay líneas de compras unidos a líneas de venta repetidas

In [21]:
ids_fact_line_repetidas = match_1ro.loc[(~match_1ro['fact_line_id'].isna()) & (match_1ro['fact_line_id'].duplicated()), 'fact_line_id']
ids_order_line_con_fact_line_repetidas = match_1ro.loc[match_1ro['fact_line_id'].isin(ids_fact_line_repetidas), 'line_id']

print(f'Son {len(ids_order_line_con_fact_line_repetidas)} líneas de compras que se unieron a líneas repetidad de fact_line_id')


Son 25 líneas de compras que se unieron a líneas repetidad de fact_line_id


Se procede a segmentar del merge 'match_1ro_fase_b' los match que sí se pueden usar y delimitas el trabajo faltante

In [22]:
match_1ro_compras_ok = match_1ro.loc[(~match_1ro['fact_line_id'].isna()) & (~match_1ro['line_id'].isin(ids_order_line_con_fact_line_repetidas)), ['fact_line_id', 'line_id']]

match_1ro_compras_faltantes = compras_fase_B.loc[~compras_fase_B['line_id'].isin(match_1ro_compras_ok['line_id'])]
match_1ro_ventas_faltantes = ventas_fase_B.loc[~ventas_fase_B['fact_line_id'].isin(match_1ro_compras_ok['fact_line_id'])]

print(f'Son un total de {len(compras_fase_B)} líneas de compra en la fase B.')

print(f'   En el 1er match se emparejaron {len(match_1ro_compras_ok)} de ellas {len(match_1ro_compras_ok)/len(compras_fase_B)*100:.2f}%')
print(f'   Quedan por emparejar {len(match_1ro_compras_faltantes)} de líneas {len(match_1ro_compras_faltantes)/len(compras_fase_B)*100:.2f}%')

Son un total de 1429 líneas de compra en la fase B.
   En el 1er match se emparejaron 735 de ellas 51.43%
   Quedan por emparejar 694 de líneas 48.57%


### ¡WEP! 2da vuelta: productos que tuvieron líneas repetidas de venta y los que no se mercharon!!

In [23]:
# For para dar tratamiento a las líneas de compra que tuvieron tuvieron match con una misma línea de venta
wep_vacias = []

ids_ventas_a_descontar = []
lista_ventas_merged = []

for i in range(len(match_1ro_compras_faltantes)):

    linea_compra = match_1ro_compras_faltantes.iloc[i]

    mini_df = match_1ro_ventas_faltantes.loc[
                ~(match_1ro_ventas_faltantes['fact_line_id'].isin(ids_ventas_a_descontar))
                & (match_1ro_ventas_faltantes['product_id'] == linea_compra['product_id_pp'])
                & (match_1ro_ventas_faltantes['salesperson_name'] == linea_compra['vendedora'])
                & (match_1ro_ventas_faltantes['quantity'] == linea_compra['product_qty'])
                & (match_1ro_ventas_faltantes['invoice_date'] >= linea_compra['order_date'] - pd.Timedelta(days=5))
                & (match_1ro_ventas_faltantes['invoice_date'] <= linea_compra['order_date'] + pd.Timedelta(days=15))
            ]

# Línea para quitar del siguiente mini_df de ventas
    if len(mini_df) == 1:
        venta = mini_df['fact_line_id'].iloc[0]
        compra = linea_compra['line_id']
        
        ids_ventas_a_descontar.append(venta)
        lista_ventas_merged.append([venta, compra])

    if len(mini_df) > 1:
        mini_df_varios = mini_df.copy()
        mini_df_varios['diff'] = abs(mini_df_varios['invoice_date'] - linea_compra['order_date'])
        venta = mini_df_varios.sort_values('diff')['fact_line_id'].iloc[0]
        compra = linea_compra['line_id']

        ids_ventas_a_descontar.append(venta)
        lista_ventas_merged.append([venta, compra])

    if mini_df.empty:
        wep_vacias.append(linea_compra['line_id'])

print(len(wep_vacias), 'wep_vacias')

match_2do_compras_ok = pd.DataFrame(lista_ventas_merged, columns=['fact_line_id', 'line_id'])
print(len(match_2do_compras_ok), 'merchadas')

634 wep_vacias
60 merchadas


In [24]:
match_2do_compras_faltantes = compras_fase_B.loc[
                                    (~compras_fase_B['line_id'].isin(match_1ro_compras_ok['line_id'])) 
                                    & (~compras_fase_B['line_id'].isin(match_2do_compras_ok['line_id']))
                                ]
match_2do_ventas_faltantes = ventas_fase_B.loc[
                                    (~ventas_fase_B['fact_line_id'].isin(match_1ro_compras_ok['fact_line_id'])) 
                                    & (~ventas_fase_B['fact_line_id'].isin(match_2do_compras_ok['fact_line_id']))
                                ]

print(f'Son un total de {len(compras_fase_B)} líneas de compra en la fase B.')

print(f'   En el 1er match se emparejaron {len(match_1ro_compras_ok)} de ellas {len(match_1ro_compras_ok)/len(compras_fase_B)*100:.2f}%')
print(f'   En el 2do match se emparejaron {len(match_2do_compras_ok)} de ellas {len(match_2do_compras_ok)/len(compras_fase_B)*100:.2f}%')
print(f'   Quedan por emparejar {len(match_2do_compras_faltantes)} de líneas {len(match_2do_compras_faltantes)/len(compras_fase_B)*100:.2f}%')

Son un total de 1429 líneas de compra en la fase B.
   En el 1er match se emparejaron 735 de ellas 51.43%
   En el 2do match se emparejaron 60 de ellas 4.20%
   Quedan por emparejar 634 de líneas 44.37%


In [25]:
# For para dar tratamiento a las líneas de compra que tuvieron tuvieron match con una misma línea de venta
wep_vacias = []

ids_ventas_a_descontar = []
lista_ventas_merged = []

for i in range(len(match_2do_compras_faltantes)):

    linea_compra = match_2do_compras_faltantes.iloc[i]

    mini_df = match_2do_ventas_faltantes.loc[
                ~(match_2do_ventas_faltantes['fact_line_id'].isin(ids_ventas_a_descontar))
                & (match_2do_ventas_faltantes['product_id'] == linea_compra['product_id_pp'])
                & (match_2do_ventas_faltantes['quantity'] == linea_compra['product_qty'])
                & (match_2do_ventas_faltantes['invoice_date'] >= linea_compra['order_date'] - pd.Timedelta(days=5))
                & (match_2do_ventas_faltantes['invoice_date'] <= linea_compra['order_date'] + pd.Timedelta(days=10))
            ]

# Línea para quitar del siguiente mini_df de ventas
    if len(mini_df) == 1:
        venta = mini_df['fact_line_id'].iloc[0]
        compra = linea_compra['line_id']
        
        ids_ventas_a_descontar.append(venta)
        lista_ventas_merged.append([venta, compra])

    if len(mini_df) > 1:
        mini_df_varios = mini_df.copy()
        mini_df_varios['diff'] = abs(mini_df_varios['invoice_date'] - linea_compra['order_date'])
        venta = mini_df_varios.sort_values('diff')['fact_line_id'].iloc[0]
        compra = linea_compra['line_id']

        ids_ventas_a_descontar.append(venta)
        lista_ventas_merged.append([venta, compra])

    if mini_df.empty:
        wep_vacias.append(linea_compra['line_id'])

print(len(wep_vacias), 'wep_vacias')

match_3ro_compras_ok = pd.DataFrame(lista_ventas_merged, columns=['fact_line_id', 'line_id'])
print(len(match_3ro_compras_ok), 'merchadas')

567 wep_vacias
67 merchadas


In [26]:
match_3ro_compras_faltantes = compras_fase_B.loc[
                                (~compras_fase_B['line_id'].isin(match_1ro_compras_ok['line_id']))
                                & (~compras_fase_B['line_id'].isin(match_2do_compras_ok['line_id']))
                                & (~compras_fase_B['line_id'].isin(match_3ro_compras_ok['line_id']))
                            ]
match_3ro_ventas_faltantes = ventas_fase_B.loc[
                                (~ventas_fase_B['fact_line_id'].isin(match_1ro_compras_ok['fact_line_id'])) 
                                & (~ventas_fase_B['fact_line_id'].isin(match_2do_compras_ok['fact_line_id']))
                                & (~ventas_fase_B['fact_line_id'].isin(match_3ro_compras_ok['fact_line_id']))
                            ]

print(f'Son un total de {len(compras_fase_B)} líneas de compra en la fase B.')

print(f'   En el 1er match se emparejaron {len(match_1ro_compras_ok)} de ellas {len(match_1ro_compras_ok)/len(compras_fase_B)*100:.2f}%')
print(f'   En el 2do match se emparejaron {len(match_2do_compras_ok)} de ellas {len(match_2do_compras_ok)/len(compras_fase_B)*100:.2f}%')
print(f'   En el 3ro match se emparejaron {len(match_3ro_compras_ok)} de ellas {len(match_3ro_compras_ok)/len(compras_fase_B)*100:.2f}%')
print(f'   Quedan por emparejar {len(match_3ro_compras_faltantes)} de líneas {len(match_3ro_compras_faltantes)/len(compras_fase_B)*100:.2f}%')

Son un total de 1429 líneas de compra en la fase B.
   En el 1er match se emparejaron 735 de ellas 51.43%
   En el 2do match se emparejaron 60 de ellas 4.20%
   En el 3ro match se emparejaron 67 de ellas 4.69%
   Quedan por emparejar 567 de líneas 39.68%


In [27]:
lista_capturistas = [
         'Elsa Ivette Diaz Leyva',
         'Alexa Yadira Mazariegos Zunun',
         'Dulce Guadalupe Pedroza Valenzuela',
         'Mariana Araceli Carvajal Flores',
         'Rosario Martinez Zarate',
      ]

match_3ro_compras_faltantes.loc[~match_3ro_compras_faltantes['vendedora'].isin(lista_capturistas)]


,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
1,20002,3307,P03292 (C 271806),2024-07-04 21:04:56,5321,El Gran Tlapalero,27394,[247181] Pinza electricista Klein tool #KT215 ...,10.00,549.32,False,C 271806,2024-06-21,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-29
3,20000,3306,P03291 (4HGFFI 253066),2024-07-04 20:28:50,5387,Home Depot Mexico,28209,[13114] Kola Loka Tubo 2Gr *13114*,4.00,18.97,False,4HGFFI 253066,2024-07-04,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-06-29
34,19949,3297,P03282 (B 87492),2024-07-04 19:06:46,5290,Maria Reyna Navarrete Cortez,13959,[5790] Triplay Pino 3/4 4'*8' 0/Caras *5790*,10.00,571.98,False,B 87492,2024-07-04,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-06-29
35,19946,3296,P03281 (42393),2024-07-04 19:01:48,5378,Maderas Dimar,26504,"[820] Riel D/Extenc. Oculto Cierre Suave 18"" 4...",12.00,164.55,False,42393,2024-07-04,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-06-29
57,19922,3286,P03271 (B 87470),2024-07-03 19:51:41,5290,Maria Reyna Navarrete Cortez,13624,[10635] Triplay Parota 1/4 *10635*,1.00,362.79,False,B 87470,2024-07-03,Elsa Ivette Diaz Leyva,Mayra Angelica Parada Manjarrez,2024-06-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13062,135,91,P00080 (C 259462),2024-01-08 23:24:24,5321,El Gran Tlapalero,13552,[2489] Chapa Ph P/Perfil Alum #550-AN #585 CE ...,5.00,327.47,False,C 259462,NaT,<NA>,Brenda Luz Acosta Lopez,2024-01-03
13066,132,89,P00078 (B 83278),2024-01-08 22:19:12,5290,Maria Reyna Navarrete Cortez,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,5.00,443.34,False,B 83278,NaT,<NA>,Mayra Angelica Parada Manjarrez,2024-01-03
13067,130,88,P00077 (IWAYYB 12428),2024-01-08 22:17:27,5281,Nueva Wal Mart De Mexico,12446,[10695] Brasso Liquido 200mls *10695*,8.00,151.72,False,IWAYYB 12428,NaT,<NA>,Irma Carvajal Flores,2024-01-03
13068,129,87,P00076 (IWAJY 697677),2024-01-08 22:15:35,5281,Nueva Wal Mart De Mexico,12446,[10695] Brasso Liquido 200mls *10695*,12.00,151.72,False,IWAJY 697677,NaT,<NA>,Irma Carvajal Flores,2024-01-03


In [33]:
wep = match_3ro_compras_faltantes.loc[~match_3ro_compras_faltantes['vendedora'].isin(lista_capturistas)]
wep.groupby('product_id_pp').count()

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
product_id_pp,,,,,,,,,,,,,,,
7481,2,2,2,2,2,2,2,2,2,2,2,0,0,2,2
7543,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1
7640,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1
7671,7,7,7,7,7,7,7,7,7,7,7,0,2,7,7
7677,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29095,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
29096,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
29241,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1


In [28]:
match_fase_B = pd.concat([match_1ro_compras_ok, match_2do_compras_ok, match_3ro_compras_ok]).rename(columns={'fact_line_id': 'fact_line_id_match','line_id': 'line_id_match' })

costo_venta_fase_B = (
    match_fase_B.merge(
        ventas_fase_B,
        how='left',
        left_on='fact_line_id_match',
        right_on='fact_line_id'
    ).merge(
        compras_fase_B,
        how='left',
        left_on='line_id_match',
        right_on='line_id'
    )
)

costo_venta_fase_B.drop(columns=['fact_line_id_match', 'line_id_match'], inplace=True)

In [29]:
ventas_merged = pd.concat([costo_venta_fase_A, costo_venta_fase_B])

In [30]:
# desktop_path = Path.home().joinpath('desktop')
# archivo_path = desktop_path.joinpath('lineas_faltantes_costo_ventas' + '.xlsx')

# writer = pd.ExcelWriter(archivo_path, engine="openpyxl")

# match_3ro_compras_faltantes.to_excel(writer, sheet_name='comp_falt')
# match_3ro_ventas_faltantes.to_excel(writer, sheet_name='vent_falt')
# ventas_merged.to_excel(writer, sheet_name='vent_merged')
# ventas_año.to_excel(writer, sheet_name='total_ventas')

# writer.close()
# writer.handles = None

# No seguir  
Se ocupa terminar el costo de ventas: estas líneas son para obtener la "Utilidad" y el "Margen de Contribución".

In [31]:
# #Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

# for i in range(len(ventas_año['price_subtotal'])):
#     ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [32]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]